In [6]:
%pip install crewai
%pip install crewai-tools


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Instantiate the tools

In [3]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### Set up the watsonx.ai instance (if this is the chosen LLM server)

In [ ]:
import os
from crewai import LLM

WATSONX_URL = "https://us-south.ml.cloud.ibm.com"
WATSONX_API_KEY = ""
WATSONX_PROJECT_ID = ""
WATSONX_MODEL_ID = "watsonx/meta-llama/llama-3-8b-instruct"
SERPER_API_KEY = ""

os.environ["WATSONX_URL"] = WATSONX_URL  # (required) Base URL of your WatsonX instance
os.environ["WATSONX_APIKEY"] = WATSONX_API_KEY  # IBM cloud API key (required)
os.environ["WATSONX_PROJECT_ID"] = WATSONX_PROJECT_ID  # Project ID of your WatsonX instance (required)
os.environ["SERPER_API_KEY"] = SERPER_API_KEY


## Model instance - Watsonx

In [ ]:
llm = LLM(
    model=WATSONX_MODEL_ID,
    base_url=WATSONX_URL,
    project_id=WATSONX_PROJECT_ID,
    max_tokens=2000,
    temperature=0.7
)

## Model instance - Ollama

In [ ]:
llm = LLM(
    model="ollama/granite3-dense:latest",
    base_url="http://localhost:11434"
)

## Define the Agents
 In the CrewAI framework, an Agent is an autonomous unit that can:

- Perform specific tasks
- Make decisions based on its role and goal
- Use tools to accomplish objectives
- Communicate and collaborate with other agents
- Maintain memory of interactions
- Delegate tasks when allowed


In [5]:
from crewai import Agent, Task, Crew, Process

data_collector = Agent(
    role='Data Collector',
    goal='Collect accurate and up-to-date financial data',
    backstory='You are an expert in gathering financial data from various sources.',
    tools=[scrape_tool, search_tool],
    verbose=True,
    allow_delegation=True,
    llm=llm
)

financial_analyst = Agent(
    role='Financial Analyst',
    goal='Analyze financial data and provide insights',
    backstory='You are a seasoned financial analyst with years of experience in interpreting market trends.',
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool],
    llm=llm
)

report_writer = Agent(
    role='Report Writer',
    goal='Compile findings into a comprehensive report',
    backstory='You are skilled at creating clear and concise financial reports.',
    llm=llm
)

## Define the tasks

In the CrewAI framework, a Task is a specific assignment completed by an Agent.

Tasks provide all necessary details for execution, such as a description, the agent responsible, required tools, and more, facilitating a wide range of action complexities. Tasks within CrewAI can be collaborative, requiring multiple agents to work together. This is managed through the task properties and orchestrated by the Crew’s process, enhancing teamwork and efficiency.

In [ ]:
data_collector_task = Task(
    description='Collect stock data for {company_name} from their company website {company_website} and yahoo finance site {yahoo_finance} for the past month',
    expected_output="""
        A list with all stock data for {company_name} from their company website {company_website} 
        and yahoo finance site {yahoo_finance} for the past month collected
    """,
    agent=data_collector
)

In [ ]:

financial_analyst_task = Task(
    description='Analyze the collected data and identify trends',
    expected_output="""
        Analyze the collected data and identify trends
    """,
    agent=financial_analyst
)
report_writer_task = Task(
    description='Write a comprehensive report on the financial analysis',
    expected_output="""
       Write a comprehensive report on the financial analysis'
    """,
    agent=report_writer
)


## Assemble the crew

A crew in crewAI represents a collaborative group of agents working together to achieve a set of tasks. Each crew defines the strategy for task execution, agent collaboration, and the overall workflow.


### Task Execution Flow
Tasks can be executed in two ways:

- Sequential: Tasks are executed in the order they are defined
- Hierarchical: Tasks are assigned to agents based on their roles and expertise

In [ ]:
financial_crew = Crew(
    agents=[data_collector, financial_analyst, report_writer],
    tasks=[data_collector_task, financial_analyst_task, report_writer_task],
    process=Process.sequential
)

# Execute the analysis
inputs = {
    'company_name': 'Tesla Inc.',
    'company_website': 'https://www.tesla.com/',
    'yahoo_finance': 'https://finance.yahoo.com/quote/TSLA'
}


## Crew Execution Process
- Sequential Process: Tasks are executed one after another, allowing for a linear flow of work.
- Hierarchical Process: A manager agent coordinates the crew, delegating tasks and validating outcomes before proceeding. Note: A manager_llm or manager_agent is required for this process and it’s essential for validating the process flow.
​
### Kicking Off a Crew
Once your crew is assembled, initiate the workflow with the kickoff() method. This starts the execution process according to the defined process flow.

In [8]:
result = financial_crew.kickoff(inputs)
print(result)

# Agent: Data Collector
## Task: Collect stock data for Tesla Inc. from their company website https://www.tesla.com/ and yahoo finance site https://finance.yahoo.com/quote/TSLA for the past month
 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.tesla.com', port=443): Read timed out. (read timeout=15).
 Tool Read website content accepts these inputs: Tool Name: Read website content
Tool Arguments: {'website_url': {'description': 'Mandatory website url to read the file', 'type': 'str'}}
Tool Description: A tool that can be used to read a website content.



# Agent: Data Collector
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.tesla.com/\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: HTTPSConnectionPool(host='www.tesla.com', port=443): Read timed out. (read timeout=15).
 Tool Read website content accepts these inputs: Tool Name: Read website conte